In [13]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [14]:

def get_title_data(soup):

    try:
      
        title = soup.find("span", attrs={"id":'productTitle'})
    
        title_value = title.text

        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_price_data(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price


def get_ratings_data(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating


def get_review_count_data(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

def get_availability_data(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [16]:

if __name__ == '__main__':
 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})

    #  URL
    URL = "https://www.amazon.com/s?k=headphones&crid=34WBR6B28SHWC&sprefix=headphone%2Caps%2C441&ref=nb_sb_noss_2"

    #http Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    dic = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list[:10]:
        new_webpg = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpg.content, "html.parser")

        # Function calls to display all necessary product information
        dic['title'].append(get_title_data(new_soup))
        dic['price'].append(get_price_data(new_soup))
        dic['rating'].append(get_ratings_data(new_soup))
        dic['reviews'].append(get_review_count_data(new_soup))
        dic['availability'].append(get_availability_data(new_soup))

    
    df = pd.DataFrame.from_dict(dic)
    df['title'].replace('', np.nan, inplace=True)
    df = df.dropna(subset=['title'])
    df.to_csv("amazon_products.csv", header=True, index=False)


ConnectionError: HTTPSConnectionPool(host='www.amazon.comhttps', port=443): Max retries exceeded with url: //aax-us-iad.amazon.com/x/c/ROn32TS7r_lLw9vt4Rq8gDEAAAGHreU07gEAAAH2AQBvbm9fdHhuX2JpZDMgICBvbm9fdHhuX2ltcDEgICBspakL/https://www.amazon.com/Eduvy-Headphones-Classroom-Elementary-Students/dp/B0B8VZPCXC/ref=sxin_26_sbv_search_btf?content-id=amzn1.sym.b000e0a0-9e93-480f-bf78-a83c8136dfcb%3Aamzn1.sym.b000e0a0-9e93-480f-bf78-a83c8136dfcb&crid=34WBR6B28SHWC&cv_ct_cx=headphones&keywords=headphones&pd_rd_i=B0B8VZPCXC&pd_rd_r=adee320e-d4e2-4d4a-a4e6-4c2c4cd69b12&pd_rd_w=39WdC&pd_rd_wg=nG0pt&pf_rd_p=b000e0a0-9e93-480f-bf78-a83c8136dfcb&pf_rd_r=TPJECXZJNGSQHXXWZ4KV&qid=1682249692&sbo=RZvfv%2F%2FHxDF%2BO5021pAnSA%3D%3D&sprefix=headphone%2Caps%2C441&sr=1-1-5190daf0-67e3-427c-bea6-c72c1df98776 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C11F30E340>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [11]:
df

,title,price,rating,reviews,availability
0,"Classroom Headphones Bulk 5 Pack, Student On E...",$37.97,4.6 out of 5 stars,"2,267 ratings",Not Available
1,2 Pack -Apple Earbuds for iPhone Headphones [A...,$18.99,4.8 out of 5 stars,855 ratings,Not Available
